<a href="https://colab.research.google.com/github/LunaMing/MagentaTrain/blob/master/Magenta_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **先开！GPU！**

# 安装magenta

In [0]:
#@test {"output": "ignore"}
%tensorflow_version 1.x

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

TensorFlow 1.x selected.
Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 12.0MB/s 
     |████████████████████████████████| 1.6MB 13.6MB/s 
     |████████████████████████████████| 2.3MB 94kB/s 
     |████████████████████████████████| 1.6MB 27.1MB/s 
     |████████████████████████████████| 204kB 55.7MB/s 
     |██

# 生成一个旋律（预训练模型）


记得导入attention_rnn.mag，就直接拖进来。

In [0]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file=/content/attention_rnn.mag \
--output_dir=/content/melody_rnn/generated \
--num_outputs=10 \
--num_steps=128 \
--primer_melody="[60]"

2020-05-18 13:35:33.147803: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-18 13:35:33.181227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-18 13:35:33.181863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-05-18 13:35:33.182175: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-18 13:35:33.183645: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-18 13:35:33.185152: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-

以“小星星”为旋律引子作曲；
就是添加了：60, -2, 60, -2, 67, -2, 67, -2

In [0]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file=/content/attention_rnn.mag \
--output_dir=/content/melody_rnn/generated \
--num_outputs=10 \
--num_steps=128 \
--primer_melody="[60, -2, 60, -2, 67, -2, 67, -2]"

2020-05-18 13:42:35.260904: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-18 13:42:35.295728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-18 13:42:35.296343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-05-18 13:42:35.296659: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-18 13:42:35.298101: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-18 13:42:35.299621: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-

# 构建数据集

新建一个mid_input文件夹，将要训练的mid文件上传

In [0]:
!convert_dir_to_note_sequences \
  --input_dir=/content/mid_input \
  --output_file=/content/notesequences.tfrecord \
  --recursive

INFO:tensorflow:Converting files in '/content/mid_input/'.
I0518 13:51:59.834984 139786549172096 convert_dir_to_note_sequences.py:64] Converting files in '/content/mid_input/'.
INFO:tensorflow:0 files converted.
I0518 13:51:59.836160 139786549172096 convert_dir_to_note_sequences.py:70] 0 files converted.
INFO:tensorflow:Converted MIDI file /content/mid_input/いろは唄.mid.
I0518 13:51:59.910540 139786549172096 convert_dir_to_note_sequences.py:134] Converted MIDI file /content/mid_input/いろは唄.mid.
INFO:tensorflow:Converted MIDI file /content/mid_input/パンツ脱げるもん！.mid.
I0518 13:51:59.960492 139786549172096 convert_dir_to_note_sequences.py:134] Converted MIDI file /content/mid_input/パンツ脱げるもん！.mid.
/usr/local/lib/python3.6/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
INFO:tensorflow:Converted MIDI file /

# 创建SequenceExamples

In [0]:
!melody_rnn_create_dataset \
--config='attention_rnn' \
--input=/content/notesequences.tfrecord \
--output_dir=/content/melody_rnn/sequence_examples \
--eval_ratio=0.10

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0518 13:53:27.227272 140339709228928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/magenta/pipelines/pipeline.py:310: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0518 13:53:28.165651 140339709228928 note_sequence_pipelines.py:174] Key signatures ignored by TranspositionPipeline.
W0518 13:53:28.844790 140339709228928 note_sequence_pipelines.py:174] Key signatures ignored by TranspositionPipeline.
W0518 13:53:29.746882 140339709228928 note_sequence_pipelines.py:174] Key signatures ignored by TranspositionPipeline.
W0518 13:53:29.747339 140339709228928 note_sequence_pipelines.py:174] Key signatures ignored by TranspositionPipeline.
W0518 13:53:29.747508 140339709228928 note_sequence_pipelines.py:174] Key signatures ignored by Trans

# 训练和验证模型

In [0]:
!melody_rnn_train \
--config=attention_rnn \
--run_dir=/content/melody_rnn/logdir/run1 \
--sequence_example_file=/content/melody_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

INFO:tensorflow:Accuracy = 0.73246086, Global Step = 1680, Loss = 0.88500774, Perplexity = 2.4230032 (12.908 sec)
I0518 14:37:35.120745 140352060356480 basic_session_run_hooks.py:260] Accuracy = 0.73246086, Global Step = 1680, Loss = 0.88500774, Perplexity = 2.4230032 (12.908 sec)
INFO:tensorflow:global_step/sec: 0.774699
I0518 14:37:35.121955 140352060356480 basic_session_run_hooks.py:692] global_step/sec: 0.774699
INFO:tensorflow:Accuracy = 0.72445226, Global Step = 1690, Loss = 0.916064, Perplexity = 2.4994333 (12.845 sec)
I0518 14:37:47.965411 140352060356480 basic_session_run_hooks.py:260] Accuracy = 0.72445226, Global Step = 1690, Loss = 0.916064, Perplexity = 2.4994333 (12.845 sec)
INFO:tensorflow:global_step/sec: 0.778558
I0518 14:37:47.966203 140352060356480 basic_session_run_hooks.py:692] global_step/sec: 0.778558
INFO:tensorflow:global_step/sec: 0.776387
I0518 14:38:00.943354 140352060356480 basic_session_run_hooks.py:692] global_step/sec: 0.776387
INFO:tensorflow:Accuracy =

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir /content/melody_rnn/logdir